In [182]:
class SuffixTree:
    def __init__(self):
        self.word = None
        self.nodeNumber = 0
        self.root = Node(0,None)
        self.activeNode = self.root;
        self.activeEdgeChar= None;
        self.activeLength = 0
        self.root.suffixLink = self.root
        self.end = -1
        self.remaining = []
        self.activePoint = None;
        self.root.parentTree = self
        self.nodeNumber=1
        
    def getActivePoint(self):
        if(self.activeLength == 0 or self.activeEdgeChar is None ):
            return Point(self.activeNode,None,0)
        activeEdge = self.activeNode.charToEdge[self.activeEdgeChar]
        activePoint = activeEdge.getPoint(self.activeEdgeChar,self.activeLength,self.remaining,0)
        self.activeNode = activePoint.node;
        self.activeEdgeChar = activePoint.edge.startChar;
        self.activeLength = activePoint.distance
        self.activePoint = activePoint;
        print("activePoint is at {0} edge at {1} distance from {2}".format(self.activeEdgeChar
                                                                           ,self.activeLength,self.activeNode))
        return activePoint
    
    def getEnd(self):
        return self.end
    
    def createLeaf(self,char):
        if(self.activeLength > 0):
            raise("cannot create leaf with {0} length",self.activeLength)
        leafNode = Node(self.nodeNumber,self)
        print("createLeaf created leaf with ",leafNode, " for char ",char)
        leafEdge = Edge(self.getEnd(),None,self.activeNode,leafNode,char)
        print("createLeaf createdEdge ",leafEdge)
        self.activeNode.charToEdge[char]=leafEdge
        
    def split(self,char,index):
        if(self.activeLength <= 0):
            raise("cannot split with {0} length",self.activeLength)
        print("creating internal node with ",self.nodeNumber)
        newInternalNode = Node(self.nodeNumber,self)
        print("creating leaf node with ",self.nodeNumber)
        leafNode = Node(self.nodeNumber,self)
        oldEdge = self.activePoint.edge ; 
        print("old edge ",oldEdge)
        internalToOldLeafChar = self.getWord()[oldEdge.startIndex + self.activeLength]
        internalToOldLeafEdge = Edge(oldEdge.startIndex + self.activeLength ,None,newInternalNode,oldEdge.endNode,internalToOldLeafChar)
        internalToNewLeafEdge = Edge(index,None,newInternalNode,leafNode,char)
        print("internalToOldLeafEdge ",internalToOldLeafEdge)
        print("internalToNewLeafEdge ",internalToNewLeafEdge)
        oldEdge.endIndex = oldEdge.startIndex + self.activeLength - 1
        oldEdge.endNode = newInternalNode
        print("oldEdge after change ",oldEdge)
        newInternalNode.charToEdge[char] = internalToNewLeafEdge ;
        newInternalNode.charToEdge[self.getWord()[oldEdge.startIndex + self.activeLength]] = internalToOldLeafEdge
        print("newInternalNode charToEdge " , newInternalNode.charToEdge)
        
        
    def __repr__(self):
        return "word {0},root {1},activeNode {2},activeEdgeChar {3},activeLength {4},end {5},remaining {6}".format(
        self.word,self.root,self.activeNode,self.activeEdgeChar,self.activeLength,self.end,self.remaining
        )
    
    def getNext(self,char):
        if(self.activeLength == 0):
            #you're currently on a node
            return self.activeNode.getNext(char)
        if(self.activePoint is not None):
            return self.activePoint.getNext(char)
        return None
    
    def getWord(self):
        return self.word

    def generateSuffixTree(self,word):
        self.word = word ;
        
        for i in range(len(self.word)):
            print("Tree ",self)
            print("i={0}".format(i))
            #rule 1
            self.end = i
            char = self.word[i]
            self.remaining.append(char)
            while(len(self.remaining) > 0):
                print("Tree",self)
                print("{0} remaining , char {1} ".format(self.remaining,char))
                activePoint = self.getActivePoint()
                print("activePoint is now ",self.getActivePoint())
                nextPoint = self.getNext(char)
                print("nextPoint ",nextPoint)
                if(nextPoint is None):
                    #rule2
                    self.remaining.pop(0)
                    if(self.activeLength == 0):
                        self.createLeaf(char)
                    else:
                        self.split(char,i)
                    if(self.activeNode == self.root):
                        self.activeLength = max(0,self.activeLength - 1)
                        self.activeEdgeChar = self.remaining[0] if len(self.remaining)>0 else None
                    self.activeNode = self.activeNode.suffixLink
                    
                else:
                    #rule3
                    self.activeLength += 1
                    if(self.activeEdgeChar is None):
                        self.activeEdgeChar = char
                    break;
class Node:
    def __init__(self,nodeNumber,parentTree):
        self.charToEdge = dict()
        self.nodeNumber = nodeNumber
        self.suffixLink = parentTree.root if parentTree is not None else None
        self.parentTree = parentTree 
        if(self.parentTree is not None):
            self.parentTree.nodeNumber += 1
        
    def getWord(self):
        return self.parentTree.word
    
    def getEnd(self):
        return self.parentTree.getEnd()
    
    def __repr__(self):
        return "Node {0}".format(self.nodeNumber)
    
    def getNext(self,char):
        if(char in self.charToEdge):
            edge = self.charToEdge[char]
            if(edge.getLength() == 1):
                return Point(edge.endNode,None,0)
            return Point(self,edge,1)
        return None;
    
    def getPoint(self,char,distance,remaining,covered):
        print("Node.getPoint called with ",char,distance,remaining,covered)
        if(char in self.charToEdge):
            edge = self.charToEdge[char]
            if(edge.getLength() == distance):
                return Point(edge.endNode,None,0)
            elif(edge.getLength() < distance):
                nextCharIndex = covered + edge.getLength() 
                print("nextCharIndex ",nextCharIndex)
                nextChar = remaining[nextCharIndex]
                return edge.endNode.getPoint(nextChar,distance - edge.getLength(),covered + edge.getLength())
            else:
                return Point(self,edge,distance)
        return None;
        
class Edge:
    def __init__(self,startIndex,endIndex,startNode,endNode,startChar):
        self.startIndex = startIndex ;
        self.endIndex = endIndex ; 
        self.startNode = startNode ;
        self.endNode = endNode ;
        self.startChar = startChar
        
    def getLength(self):
        if(self.endIndex is None):
            return self.getEnd()-self.startIndex + 1;
        return self.endIndex - self.startIndex + 1
    
    def getEnd(self):
        if(self.endIndex is not None):
            return self.endIndex
        return self.startNode.getEnd()
    
    def getWord(self):
        return self.startNode.getWord()
    
    def getPoint(self,activeEdgeChar,activeLength,remaining,covered):
        print("Edge.getPoint called with ",activeEdgeChar,activeLength,remaining,covered)
        edgeLength = self.getLength()
        if(edgeLength == activeLength):
            return Point(self.endNode,None,0)
        elif(edgeLength < activeLength):
            nextCharIndex = edgeLength + covered ;
            print("nextCharIndex ",nextCharIndex)
            charFromNode = remaining[nextCharIndex]
            return self.endNode.getPoint(charFromNode,activeLength - edgeLength,remaining,covered+edgeLength)
        else:
            return Point(self.startNode, self,activeLength)
        
    def __repr__(self):
        return "Edge {0}..{1} ,{2} --> {3} of length {4} starting with {5}".format(self.startIndex,self.getEnd(),
                                                                                  self.startNode,self.endNode,self.getLength()
                                                                                   ,self.startChar
                                                                                  )
class Point:
    def __init__(self,node,edge,distance):
        self.node = node ;
        self.edge = edge;
        self.distance = distance
        
    def getWord(self):
        return self.node.getWord()
    
    #method only for 1 lookup ahead
    def getNext(self,char):
        edgeLength = self.edge.getLength()
        distance = self.distance
        if(edgeLength == distance):
            return self.edge.endNode.getNext(char)
        nextIndex = self.edge.startIndex + distance 
        nextChar = self.getWord()[nextIndex];
        print("Point.getNext() nextIndex {0},nextChar{1},searchingForChar{2}".format(nextIndex,nextChar,char))
        if(nextChar == char):
            if(distance == self.edge.getLength() -1 ):
                return Point(self.edge.endNode,None,0)
            return Point(self.edge.startNode,self.edge,distance+1)
        return None
    
    def __repr__(self):
        return "Point at distance {0} at edge {1} from {2}".format(self.distance,self.edge,self.node)
    
tree = SuffixTree()
tree.generateSuffixTree("mississippi$")

Tree  word mississippi$,root Node 0,activeNode Node 0,activeEdgeChar None,activeLength 0,end -1,remaining []
i=0
Tree word mississippi$,root Node 0,activeNode Node 0,activeEdgeChar None,activeLength 0,end 0,remaining ['m']
['m'] remaining , char m 
activePoint is now  Point at distance 0 at edge None from Node 0
nextPoint  None
createLeaf created leaf with  Node 1  for char  m
createLeaf createdEdge  Edge 0..0 ,Node 0 --> Node 1 of length 1 starting with m
Tree  word mississippi$,root Node 0,activeNode Node 0,activeEdgeChar None,activeLength 0,end 0,remaining []
i=1
Tree word mississippi$,root Node 0,activeNode Node 0,activeEdgeChar None,activeLength 0,end 1,remaining ['i']
['i'] remaining , char i 
activePoint is now  Point at distance 0 at edge None from Node 0
nextPoint  None
createLeaf created leaf with  Node 2  for char  i
createLeaf createdEdge  Edge 1..1 ,Node 0 --> Node 2 of length 1 starting with i
Tree  word mississippi$,root Node 0,activeNode Node 0,activeEdgeChar None,activ

AttributeError: 'NoneType' object has no attribute 'startChar'